本文主要以kaggle一次二分类竞赛数据为例，深入研究改进forest

LayerDTree是LayerForest的精简版，是对Forest进一步探索的基础

- 数据地址：https://www.kaggle.com/c/porto-seguro-safe-driver-prediction
- 数据特点：
  - 非常不均衡、缺失值严重、二分类
- 包含内容：
  1. 数据获取
  2. 模型应用
  3. 结果分析

# Stage-1：获取数据

In [1]:
import os
import numpy as np
import pandas as pd
import time

In [2]:
df_train = pd.read_csv('./datasetes/driver/train.csv')
df_test = pd.read_csv('./datasetes/driver/test.csv')

In [3]:
print(df_train[df_train.target==1].shape)
print(df_train[df_train.target==0].shape)
df_train[df_train.target==1].shape[0]/df_train[df_train.target==0].shape[0]

(21694, 59)
(573518, 59)


0.037826188541597645

In [4]:
target_train = df_train['target'].values
id_train = df_train['id'].values
id_test = df_test['id'].values

In [5]:
df_train = df_train.drop(['target','id'],axis=1)
df_test = df_test.drop(['id'], axis = 1)
# combine = pd.concat([df_train,df_test],axis=0)

In [6]:
print ("The train shape is:",df_train.shape)
print ('The test shape is:',df_test.shape)
# print ('The combine shape is:',combine.shape)

The train shape is: (595212, 57)
The test shape is: (892816, 57)


# Stage-2：模型应用

In [51]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import ForestUtils
import time
import random

In [12]:
import EnhancedDTree
import importlib

In [84]:
importlib.reload(EnhancedDTree)

<module 'EnhancedDTree' from 'C:\\github_workspace\\LayerForest\\EnhancedDTree.py'>

In [75]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_metrix(a, p):
    return "nor gini:", gini_normalized(a,p)

In [82]:
train = np.array(df_train)
test = np.array(df_test)
X = train
y = target_train

X_test = test
test_y = np.array(([0.0] * len(test)))
all_data_mask = np.array([False] * len(test))

# 不均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 100
layer = 0

enhancedDTree = EnhancedDTree.EnhancedDTree()
counter = 0
while 1:
    layer += 1
    print()
    print("layer:", layer)
    X = X_train_np
    y = y_train_np
    
    # 均衡数据
    positive_mask = np.where(y == 1)[0]
    negative_index = np.where(y == 0)[0]
    random.shuffle(negative_index)
    negative_mask = negative_index[:len(positive_mask)]
    train_mask = np.hstack((positive_mask, negative_mask))
    train_data_x = X[train_mask]
    train_data_y = y[train_mask]
    guest_mask = negative_index[len(positive_mask):]
    guest_data_x = X[guest_mask]
    guest_data_y = y[guest_mask]
    
    print("train==1", train_data_y[train_data_y == 1].shape)
    print("train==0", train_data_y[train_data_y == 0].shape)
    clf, data_mask, all_false_data_index, p_test  = \
        enhancedDTree.TrainModelLayer(train_data_x, train_data_y, X_test, all_data_mask, test_y, verbose=True, feval=gini_metrix)
        
    X_train_np = enhancedDTree.X_train_np
    y_train_np = enhancedDTree.y_train_np
    
    # 均衡数据
    X_train_np = np.vstack((X_train_np, guest_data_x))
    y_train_np = np.hstack((y_train_np, guest_data_y))
    print("train==1", y_train_np[y_train_np == 1].shape)
    print("train==0", y_train_np[y_train_np == 0].shape)
    
    if X_train_np.shape[0] < 1000 or layer > maxlayer or y_train_np[y_train_np==1].shape[0] <= 10:
        all_data_mask = all_false_data_index[~data_mask]
        test_y[all_data_mask] = p_test[~data_mask]
        print(len(p_test[~data_mask]))
        break
        
    all_data_mask[~all_data_mask] = data_mask


layer: 1
train==1 (21694,)
train==0 (21694,)
X_train.shape, y_train.shape:(36879, 57)(36879,)
X_valid.shape, y_valid.shape:(6509, 57)(6509,)
data  ('nor gini:', 0.31195751556963142)
data  ('nor gini:', 0.30856039477769154)
data  ('nor gini:', 0.22942454377313704)
mean of all impurity:0.480591867043
pass node id shape:(191,)
pass data shape:(20224, 57) (10265,)
not pass data shape:(23164, 57) (265584,)
all data shape:43388
mean of all impurity:0.461183734086
pass node id shape:(128,)
pass test data shape: 285905
not pass test data shape: 606911
train==1 (11429,)
train==0 (563559,)

layer: 2
train==1 (11429,)
train==0 (11429,)
X_train.shape, y_train.shape:(19429, 57)(19429,)
X_valid.shape, y_valid.shape:(3429, 57)(3429,)
data  ('nor gini:', 0.36853755025922469)
data  ('nor gini:', 0.36667574779719025)
data  ('nor gini:', 0.23927457297304658)
mean of all impurity:0.474448849622
pass node id shape:(107,)
pass data shape:(13993, 57) (6938,)
not pass data shape:(8865, 57) (101754,)
all data

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1547,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 17
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1574,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1585,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 33
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1566,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1558,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 49
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1565,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1535,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 65
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1569,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1548,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 81
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1569,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1573,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass test data shape: 23928
train==1 (67,)
train==0 (552095,)

layer: 97
train==1 (67,)
train==0 (67,)
X_train.shape, y_train.shape:(113, 57)(113,)
X_valid.shape, y_valid.shape:(21, 57)(21,)
data  ('nor gini:', 1.0)
data  ('nor gini:', 0.043859649122806994)
data  ('nor gini:', -0.30909090909090892)
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass data shape:(0, 57) (0,)
not pass data shape:(134, 57) (1560,)
all data shape:134
mean of all impurity:0.499960842666
pass node id shape:(0,)
pass test data shape: 0
not pass 

In [73]:
X_train_np[y_train_np == 1]

array([[ 3.,  1.,  7., ...,  0.,  1.,  0.],
       [ 3.,  1.,  7., ...,  0.,  0.,  1.],
       [ 4.,  1.,  7., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  2.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  4., ...,  0.,  0.,  0.],
       [ 0.,  1.,  2., ...,  0.,  0.,  0.]])

In [80]:
np.max(clf.tree_.impurity)

0.49940546967895361

In [86]:
test_y[test_y == 0].shape

(5160,)

In [87]:
sub = pd.DataFrame(test_y, columns=['target'])
sub['id'] = id_test
sub.head()

,target,id
0,0.016134,0
1,0.025643,1
2,0.029184,2
3,0.022739,3
4,0.044334,4


In [88]:
import datetime
file_pre = datetime.datetime.now().strftime('%m_%d_%H_%M')
sub.to_csv("./submission/LayerDTree_"+file_pre+".csv", index=False)

In [85]:
train = np.array(df_train)
test = np.array(df_test)
X = train
y = target_train

X_test = test
test_y = np.array(([0.0] * len(test)))
all_data_mask = np.array([False] * len(test))

# 不均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 100
layer = 0

enhancedDTree = EnhancedDTree.EnhancedDTree()
counter = 0
while 1:
    layer += 1
    print()
    print("layer:", layer)
    X = X_train_np
    y = y_train_np
    clf, data_mask, all_false_data_index, p_test  = \
        enhancedDTree.TrainModelLayer(X, y, X_test, all_data_mask, test_y, verbose=True, feval=gini_metrix)
    X_train_np = enhancedDTree.X_train_np
    y_train_np = enhancedDTree.y_train_np
    
    if X_train_np.shape[0] < 1000 or layer > maxlayer or y_train_np[y_train_np==1].shape[0] <= 10:
        all_data_mask = all_false_data_index[~data_mask]
        test_y[all_data_mask] = p_test[~data_mask]
        print(len(p_test[~data_mask]))
        break
        
    all_data_mask[~all_data_mask] = data_mask


layer: 1
X_train.shape, y_train.shape:(505930, 57)(505930,)
X_valid.shape, y_valid.shape:(89282, 57)(89282,)
data  ('nor gini:', 0.29861587616817897)
data  ('nor gini:', 0.31669028062603555)
data  ('nor gini:', 0.19587163470940372)
mean of all impurity:0.058268339054
pass node id shape:(177,)
pass data shape:(324228, 57) (7501,)
not pass data shape:(270984, 57) (3069911,)
all data shape:595212
mean of all impurity:0.116536678108
pass node id shape:(466,)
pass test data shape: 806137
not pass test data shape: 86679

layer: 2
X_train.shape, y_train.shape:(230336, 57)(230336,)
X_valid.shape, y_valid.shape:(40648, 57)(40648,)
data  ('nor gini:', 0.25525622319418728)
data  ('nor gini:', 0.28015083120197132)
data  ('nor gini:', 0.11382729007934944)
mean of all impurity:0.0723301649046
pass node id shape:(88,)
pass data shape:(95736, 57) (3209,)
not pass data shape:(175248, 57) (1996791,)
all data shape:270984
mean of all impurity:0.144660329809
pass node id shape:(336,)
pass test data shape

data  ('nor gini:', 0.21282056086486159)
data  ('nor gini:', 0.2623674102234263)
data  ('nor gini:', -0.063987178794590732)
mean of all impurity:0.268834624603
pass node id shape:(6,)
pass data shape:(749, 57) (124,)
not pass data shape:(7157, 57) (79383,)
all data shape:7906
mean of all impurity:0.388655370462
pass node id shape:(40,)
pass test data shape: 26
not pass test data shape: 43

layer: 18
X_train.shape, y_train.shape:(6083, 57)(6083,)
X_valid.shape, y_valid.shape:(1074, 57)(1074,)
data  ('nor gini:', 0.22692004304833524)
data  ('nor gini:', 0.26475932331723645)
data  ('nor gini:', 0.0093933363071750289)
mean of all impurity:0.286035824358
pass node id shape:(4,)
pass data shape:(566, 57) (91,)
not pass data shape:(6591, 57) (73053,)
all data shape:7157
mean of all impurity:0.395216592967
pass node id shape:(35,)
pass test data shape: 13
not pass test data shape: 30

layer: 19
X_train.shape, y_train.shape:(5602, 57)(5602,)
X_valid.shape, y_valid.shape:(989, 57)(989,)
data  ('

### Todo list
- 树结构设计（完成）
- 通过gini对数据分割（完成）
- 全局测试集
- 输出结果集
- 打印信息增加pass data的比例
- 防止过拟合
- 对pass data的进一步处理
- 先进行数据均衡化是不是更快一些